In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer

from collections import Counter
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit
from sklearn import tree
from sklearn.svm import LinearSVC
from sklearn.utils import shuffle
import nltk
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize

stemmer = SnowballStemmer("english", ignore_stopwords=True)
lemmatizer = WordNetLemmatizer()
stopwords = stopwords.words('english')
np.set_printoptions(threshold=np.nan)

In [2]:
# Load data
IMDB_train = pd.read_csv('./IMDB-train.txt', sep='\t', encoding='latin-1', header=None)
IMDB_train_y = IMDB_train[:][1]
IMDB_valid = pd.read_csv('./IMDB-valid.txt', sep='\t', encoding='latin-1', header=None)
IMDB_valid_y = IMDB_valid[:][1]
IMDB_test = pd.read_csv('./IMDB-test.txt', sep='\t', encoding='latin-1', header=None)
IMDB_test_y = IMDB_test[:][1]
stemmer = SnowballStemmer("english", ignore_stopwords=True)

print("Data loaded.")

Data loaded.


In [3]:
len(IMDB_train)

15000

In [4]:
len(IMDB_valid)

10000

In [5]:
len(IMDB_test)

25000

# Preprocessing

In [6]:
def preprocessing(data):
    new_data = []
    #i = 0
    for sentence in (data[:][0]):
        #clean = re.compile('<.*?>')
        new_sentence = re.sub('<.*?>', '', sentence) # remove HTML tags
        new_sentence = re.sub(r'[^\w\s]', '', new_sentence) # remove punctuation
        new_sentence = new_sentence.lower() # convert to lower case
        if new_sentence != '':
            new_data.append(new_sentence)
    return new_data

In [7]:
"""
<<<<IGNORE THESE FOR NOW>>>>

def tokenize(data):
    new_data = []
    for sentence in (data):
        new_sentence = nltk.word_tokenize(sentence)
        new_data.append(new_sentence)
    return new_data        

def stem_lem(data):
    new_data = []
    for sent in data:
        this_sent = []
        for w in test3:
            w = stemmer.stem(w)
            w = lemmatizer.lemmatize(w)
            this_sent.append(stemmer.stem(w))
        new_data.append(this_sent)
    return new_data

def rm_stopwords(data):
    new_data = []
    for sent in data:
        new_data.append([w for w in sent if w not in stopwords])
    return new_data
"""

'\n<<<<IGNORE THESE FOR NOW>>>>\n\ndef tokenize(data):\n    new_data = []\n    for sentence in (data):\n        new_sentence = nltk.word_tokenize(sentence)\n        new_data.append(new_sentence)\n    return new_data        \n\ndef stem_lem(data):\n    new_data = []\n    for sent in data:\n        this_sent = []\n        for w in test3:\n            w = stemmer.stem(w)\n            w = lemmatizer.lemmatize(w)\n            this_sent.append(stemmer.stem(w))\n        new_data.append(this_sent)\n    return new_data\n\ndef rm_stopwords(data):\n    new_data = []\n    for sent in data:\n        new_data.append([w for w in sent if w not in stopwords])\n    return new_data\n'

In [8]:
IMDB_train = preprocessing(IMDB_train)
IMDB_valid = preprocessing(IMDB_valid)
IMDB_test = preprocessing(IMDB_test)

In [9]:
#IMDB_train_tok = tokenize(IMDB_train)
#IMDB_valid_tok = tokenize(IMDB_valid)
#IMDB_test_tok = tokenize(IMDB_test)

In [10]:
#IMDB_train_sl = stem_lem(IMDB_train_tok)
#IMDB_valid_sl = stem_lem(IMDB_valid_tok)
#IMDB_test_sl = stem_lem(IMDB_test_tok)

In [11]:
#IMDB_train_stop = rm_stopwords(IMDB_train_sl)
#IMDB_valid_stop = rm_stopwords(IMDB_valid_sl)
#IMDB_test_stop = rm_stopwords(IMDB_test_sl)

# Bag of n-gram 

In [12]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

In [13]:
"""
unigram = CountVectorizer(tokenizer=LemmaTokenizer(), analyzer='word', ngram_range=(1, 1), stop_words=None)
bigram = CountVectorizer(tokenizer=LemmaTokenizer(), analyzer='word', ngram_range=(1, 2), stop_words=None)

train_unigram = unigram.fit_transform(IMDB_train).toarray()
train_bigram = bigram.fit_transform(IMDB_train).toarray()
"""

"\nunigram = CountVectorizer(tokenizer=LemmaTokenizer(), analyzer='word', ngram_range=(1, 1), stop_words=None)\nbigram = CountVectorizer(tokenizer=LemmaTokenizer(), analyzer='word', ngram_range=(1, 2), stop_words=None)\n\ntrain_unigram = unigram.fit_transform(IMDB_train).toarray()\ntrain_bigram = bigram.fit_transform(IMDB_train).toarray()\n"

In [14]:
### RUN THIS ON SVM
# TRIGRAM TOP 30K frequent words
trigram = CountVectorizer(tokenizer=LemmaTokenizer(), analyzer='word', ngram_range=(1, 3), stop_words='english', max_features =30000)

In [ ]:
train_trigram = trigram.fit_transform(IMDB_train).toarray()
valid_trigram = trigram.fit_transform(IMDB_valid).toarray()
test_trigram = trigram.fit_transform(IMDB_test).toarray()

In [ ]:
"""
pd.DataFrame.from_records(train_trigram).to_csv('./train_trigram.csv', index = False, header = False)
pd.DataFrame.from_records(valid_trigram).to_csv('./valid_trigram.csv', index = False, header = False)
pd.DataFrame.from_records(test_trigram).to_csv('./test_trigram.csv', index = False, header = False)
"""